In [ ]:
import pandas as pd
import os
import re
import glob
import json
from tqdm import tqdm

In [ ]:
json_dir = 'C:/Users/chsze/Desktop/Georgia Tech/First Semester/DVA/Project/world_countries.json'
with open(json_dir) as f:
    data = json.load(f)
countries = [d['properties']['name'] for d in data['features']]

In [ ]:
data_dir = 'C:/Users/chsze/Desktop/Georgia Tech/First Semester/DVA/Project/archive/'
new_data_dir = os.path.join(data_dir, 'cleaned')
if not os.path.exists(new_data_dir):
    os.makedirs(new_data_dir)
files = [data_dir + file for file in os.listdir(data_dir)]

In [ ]:
contractions = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [ ]:
def clean_text(text):
    text = text.lower()
    for word in text.split():
        if word in contractions:
            text = text.replace(word, contractions[word])
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

def clean(file):
    new_file = os.path.join(new_data_dir, file.split('/')[-1])
    if os.path.isfile(new_file):
        return
    df = pd.read_csv(file)
    df = df[df.language=='en']
    cols = ['userid', 'username', 'acctdesc', 'location', 'following', 'followers', 'tweetcreatedts', 'retweetcount', 'text', 'hashtags']
    df = df.dropna(subset=cols)
    df = df[cols]
    df['acctdesc'] = df['acctdesc'].map(lambda text : clean_text(text))
    df['text'] = df['text'].map(lambda text : clean_text(text))
    df = df[df["location"].str.contains('|'.join(countries)) == True]
    df.reset_index(drop=True, inplace=True)
    df['location'] = df['location'].str.extract(f'({"|".join(countries)})', expand=False)
    df['tweetcreatedts'] =  pd.to_datetime(df['tweetcreatedts'], format='%Y-%m-%d %H:%M:%S')
    pat = "text': \'([A-Za-z0-9]+)\'"
    pat = re.compile(pat)
    hashtags = []
    for i in range(len(df)):
        text = df['hashtags'][i]
        hashtags.append(pat.findall(df['hashtags'][i]))
    df['hashtags'] = hashtags
    df.to_csv(new_file)

In [ ]:
for file in tqdm(files):
    if not os.path.isdir(file):
        clean(file)

  0%|                                                                                          | 0/232 [00:00<?, ?it/s]C:\Users\chsze\anaconda3\envs\ml_proj\lib\site-packages\ipykernel_launcher.py:3: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  This is separate from the ipykernel package so we can avoid doing imports until
100%|████████████████████████████████████████████████████████████████████████████████| 232/232 [06:56<00:00,  1.80s/it]
